In [ ]:
!curl -O http://www.manythings.org/anki/fra-eng.zip
!unzip fra-eng.zip
!head -n 100000 fra.txt | shuf | split -l 5000
!pip install tensorflow-addons

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 6562k  100 6562k    0     0  10.7M      0 --:--:-- --:--:-- --:--:-- 10.7M
Archive:  fra-eng.zip
  inflating: _about.txt              
  inflating: fra.txt                 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 14.7 MB/s eta 0:00:00


In [ ]:
"""
Title: Character-level recurrent sequence-to-sequence model
Author: [fchollet](https://twitter.com/fchollet)
Date created: 2017/09/29
Last modified: 2020/04/26
Description: Character-level recurrent sequence-to-sequence model.
Accelerator: GPU
"""
"""
## Introduction

This example demonstrates how to implement a basic character-level
recurrent sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

**Summary of the algorithm**

- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
"""

"""
## Setup
"""

import re
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow import keras

"""
## Download the data
"""

"""shell
!curl -O http://www.manythings.org/anki/fra-eng.zip
!unzip fra-eng.zip
!split -l 10000 fra.txt
!zip -r s2s.zip s2s
"""

"""
## Configuration
"""

############################################

'\n## Configuration\n'

In [ ]:
########################################################################################
batch_size = 64  # Batch size for training.
epochs = 60  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 200000  # Number of samples to train on.
# Path to the data txt file on disk.
"""
## Prepare the data
"""

########################################

'\n## Prepare the data\n'

In [ ]:
############################################################################################

# Get full data dictionary
input_texts = []
target_texts = []
input_words = set()
target_words = set()
data_path = "fra.txt"
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    input_text =  [w for w in re.split("[^a-z0-9áàÀâÂçÇéÉèêÊëîïôÔœùûü']", input_text.lower()) if w != '']
    target_text = ['<BOS>'] + [w for w in re.split("[^a-z0-9áàÀâÂçÇéÉèêÊëîïôÔœùûü]", target_text.lower()) if w != ''] + ['<EOS>']
    input_texts.append(input_text)
    target_texts.append(target_text)
    for word in input_text:
        if word not in input_words:
            input_words.add(word)
    for word in target_text:
        if word not in target_words:
            target_words.add(word)

input_words = sorted(list(input_words))
target_words = sorted(list(target_words))
input_token_index = dict([(char, i) for i, char in enumerate(input_words)])
target_token_index = dict([(char, i) for i, char in enumerate(target_words)])
num_encoder_tokens = len(input_words)
num_decoder_tokens = len(target_words)


####################################

In [ ]:
################################################################################################
# Vectorize the data.
data_path = "xab"
num_samples = 5000
input_texts = []
target_texts = []
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    input_text =  [w for w in re.split("[^a-z0-9áàÀâÂçÇéÉèêÊëîïôÔœùûü']", input_text.lower()) if w != '']
    target_text = ['<BOS>'] + [w for w in re.split("[^a-z0-9áàÀâÂçÇéÉèêÊëîïôÔœùûü]", target_text.lower()) if w != ''] + ['<EOS>']
    input_texts.append(input_text)
    target_texts.append(target_text)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

"""
### Fill np arrays with the data
"""
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    # encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

####################################

Number of samples: 5000
Number of unique input tokens: 15910
Number of unique output tokens: 25904
Max sequence length for inputs: 8
Max sequence length for outputs: 15


In [ ]:
################################################################################################
"""
## Build the model
"""

# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
#################################################

In [ ]:
###################################################################################
# Vectorize the data.
input_texts = []
target_texts = []
input_words = set()
target_words = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    input_text =  [w for w in re.split("[^a-z0-9áàÀâÂçÇéÉèêÊëîïôÔœùûü']", input_text.lower()) if w != '']
    target_text = ['<BOS>'] + [w for w in re.split("[^a-z0-9áàÀâÂçÇéÉèêÊëîïôÔœùûü]", target_text.lower()) if w != ''] + ['<EOS>']
    input_texts.append(input_text)
    target_texts.append(target_text)
    for word in input_text:
        if word not in input_words:
            input_words.add(word)
    for word in target_text:
        if word not in target_words:
            target_words.add(word)

max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
)
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

"""
### Fill np arrays with the data
"""
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    # encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

################################################

Number of samples: 5000
Number of unique input tokens: 15910
Number of unique output tokens: 25904
Max sequence length for inputs: 8
Max sequence length for outputs: 15


In [ ]:
####################################################################################
"""
## Build the model
"""

# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)

# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

"""
## Train the model
"""

#####################################################

'\n## Train the model\n'

In [ ]:
###############################################################################

def compile():
    model.compile(
        optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
    )
compile()
####################################################

In [ ]:
################################################################################

def train():
    model.fit(
        [encoder_input_data, decoder_input_data],
        decoder_target_data,
        batch_size=batch_size,
        epochs=epochs,
        validation_split=0.2,
    )

    # Save model

train()

################################################

Epoch 1/100
63/63 [==============================] - 18s 167ms/step - loss: 2.8569 - accuracy: 0.0812 - val_loss: 2.6140 - val_accuracy: 0.0848
Epoch 2/100
63/63 [==============================] - 7s 107ms/step - loss: 2.5084 - accuracy: 0.0858 - val_loss: 2.6197 - val_accuracy: 0.0861
Epoch 3/100
63/63 [==============================] - 7s 106ms/step - loss: 2.4612 - accuracy: 0.0864 - val_loss: 2.6317 - val_accuracy: 0.0871
Epoch 4/100
63/63 [==============================] - 7s 108ms/step - loss: 2.4340 - accuracy: 0.0882 - val_loss: 2.6284 - val_accuracy: 0.0919
Epoch 5/100
63/63 [==============================] - 7s 107ms/step - loss: 2.4149 - accuracy: 0.0909 - val_loss: 2.6142 - val_accuracy: 0.0920
Epoch 6/100
63/63 [==============================] - 7s 106ms/step - loss: 2.3970 - accuracy: 0.0935 - val_loss: 2.6070 - val_accuracy: 0.0916
Epoch 7/100
63/63 [==============================] - 7s 106ms/step - loss: 2.3795 - accuracy: 0.0955 - val_loss: 2.5987 - val_accuracy: 0.096

In [ ]:
model.save("s2s")

In [ ]:
!zip -r s2s.zip s2s

  adding: s2s/ (stored 0%)
  adding: s2s/variables/ (stored 0%)
  adding: s2s/variables/variables.index (deflated 62%)
  adding: s2s/variables/variables.data-00000-of-00001 (deflated 45%)
  adding: s2s/assets/ (stored 0%)
  adding: s2s/saved_model.pb (deflated 91%)
  adding: s2s/keras_metadata.pb (deflated 90%)


In [ ]:
!unzip s2s.zip

Archive:  s2s.zip
   creating: s2s/
   creating: s2s/variables/
  inflating: s2s/variables/variables.index  
  inflating: s2s/variables/variables.data-00000-of-00001  
   creating: s2s/assets/
  inflating: s2s/saved_model.pb      
  inflating: s2s/keras_metadata.pb   


In [ ]:
model = keras.models.load_model("s2s")

In [ ]:
####################################################################################

"""
## Run inference (sampling)

1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state
and a "start of sequence" token as target.
Output will be the next target token.
3. Repeat with the current target token and current states
"""

# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model("s2s")

encoder_inputs = model.input[0]  # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]  # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


######################################

################################################

In [ ]:
##############################################
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["<BOS>"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence.append(sampled_char)

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "<EOS>" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

####################

####################################

In [ ]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["<BOS>"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = []
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence.append(sampled_char)

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "<EOS>" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        # Update states
        states_value = [h, c]
    return decoded_sentence

In [ ]:
############################################################################

# print(tfa.seq2seq.BeamSearchDecoder(cell=decoder_lstm, beam_width=10, output_layer=decoder_dense))

####################################

In [ ]:
################################################################################################

"""
You can now generate decoded sentences as such:
"""

def test():
    for seq_index in range(2000, 2020):
        # Take one sequence (part of the training set)
        # for trying out decoding.
        input_seq = encoder_input_data[seq_index : seq_index + 1]
        decoded_sentence = decode_sequence(input_seq)
        print("-")
        print("Input sentence:", input_texts[seq_index])
        print("Decoded sentence:", decoded_sentence)

test()

1/1 [==============================] - 0s 18ms/step
-
Input sentence: ['i', 'want', 'to', 'live', 'in', 'the', 'city']
Decoded sentence: ['je', 'veux', 'vivre', 'de', '<EOS>']
1/1 [==============================] - 0s 17ms/step
-
Input sentence: ['i', 'have', 'a', 'headache']
Decoded sentence: ['j', 'ai', 'un', 'de', 'de', '<EOS>']
1/1 [==============================] - 0s 17ms/step
-
Input sentence: ['the', 'game', "isn't", 'over']
Decoded sentence: ['la', 'partie', 'n', 'est', 'pas', 'pas', '<EOS>']
1/1 [==============================] - 0s 18ms/step
-
Input sentence: ['he', 'can', 'swim', 'well']
Decoded sentence: ['il', 'nage', 'le', '<EOS>']
1/1 [==============================] - 0s 17ms/step
-
Input sentence: ['tom', 'can', 'come', 'back', 'tomorrow']
Decoded sentence: ['tom', 'peut', 'revenir', '<EOS>']
1/1 [==============================] - 0s 18ms/step
-
Input sentence: ['i', 'thought', "we'd", 'be', 'safe', 'here']
Decoded sentence: ['je', 'pensais', 'que', 'nous', 'serions',

In [ ]:
encoder_input_data.shape

(5000, 8, 15910)

In [ ]:
tfa.seq2seq.BeamSearchDecoder(cell=decoder_lstm,beam_width=10,output_layer=decoder_dense)

TypeError: ignored

In [ ]:
def returntab(test_path,nMot,num_encoder_tokens,input_token_index,target_token_index):

    batch_size = 64  # Batch size for training.
    epochs = 2  # Number of epochs to train for.
    latent_dim = 256  # Latent dimensionality of the encoding space.
    num_samples = 10000  # Number of samples to train on.
    # Path to the data txt file on disk.
    data_path = "fra.txt"

    """
    ## Prepare the data
    """

    # Vectorize the data.
    input_texts = []
    target_texts = []
    input_words = set()
    target_words = set()
    with open(data_path, "r", encoding="utf-8") as f:
        lines = f.read().split("\n")
    for line in lines[: min(num_samples, len(lines) - 1)]:
        input_text, target_text, _ = line.split("\t")
        input_text =  [w for w in re.split("[^a-z0-9áàÀâÂçÇéÉèêÊëîïôÔœùûü']", input_text.lower()) if w != '']
        target_text = ['<BOS>'] + [w for w in re.split("[^a-z0-9áàÀâÂçÇéÉèêÊëîïôÔœùûü]", target_text.lower()) if w != ''] + ['<EOS>']
        input_texts.append(input_text)
        target_texts.append(target_text)
        for word in input_text:
            if word not in input_words:
                input_words.add(word)
        for word in target_text:
            if word not in target_words:
                target_words.add(word)

    max_encoder_seq_length = max([len(txt) for txt in input_texts])
    max_decoder_seq_length = max([len(txt) for txt in target_texts])


    encoder_input_data = np.zeros(
        (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
    )
    decoder_input_data = np.zeros(
        (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
    )
    decoder_target_data = np.zeros(
        (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
    )

    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.0
        # encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
        for t, char in enumerate(target_text):
            # decoder_target_data is ahead of decoder_input_data by one timestep
            decoder_input_data[i, t, target_token_index[char]] = 1.0
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

    """
    ## Build the model
    """

    # Define an input sequence and process it.
    encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
    encoder = keras.layers.LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)

    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the
    # return states in the training model, but we will use them in inference.
    decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
    decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

    """
    ## Train the model
    """

    model.compile(
        optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
    )










    

    input_texts = []


    with open(test_path, "r", encoding="utf-8") as f:
        lines = f.read().split("\n")
    for line in lines:
        input_text= line
        input_text =  [w for w in re.split("[^a-z0-9áàÀâÂçÇéÉèêÊëîïôÔœùûü']", input_text.lower()) if w != '']
        input_texts.append(input_text)
        for word in input_text:
            if word not in input_words:
                #input_words.add(word)
                print("error :" + word)



    num_encoder_tokens = len(input_words)
    max_encoder_seq_length = max([len(txt) for txt in input_texts])


    input_token_index = dict([(char, i) for i, char in enumerate(input_words)])


    encoder_input_data = np.zeros(
        (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")


    for i, input_text in enumerate(input_texts):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.0
    










    """
    ## Run inference (sampling)

    1. encode input and retrieve initial decoder state
    2. run one step of decoder with this initial state
    and a "start of sequence" token as target.
    Output will be the next target token.
    3. Repeat with the current target token and current states
    """

    # Define sampling models
    # Restore the model and construct the encoder and decoder.
    model = keras.models.load_model("s2s")

    encoder_inputs = model.input[0]  # input_1
    encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output  # lstm_1
    encoder_states = [state_h_enc, state_c_enc]
    encoder_model = keras.Model(encoder_inputs, encoder_states)

    decoder_inputs = model.input[1]  # input_2
    decoder_state_input_h = keras.Input(shape=(latent_dim,))
    decoder_state_input_c = keras.Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_lstm = model.layers[3]
    decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs
    )
    decoder_states = [state_h_dec, state_c_dec]
    decoder_dense = model.layers[4]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = keras.Model(
        [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
    )

    # Reverse-lookup token index to decode sequences back to
    # something readable.
    reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
    reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())


    def decode_sequence(input_seq):
        # Encode the input as state vectors.
        states_value = encoder_model.predict(input_seq)

        # Generate empty target sequence of length 1.
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        # Populate the first character of target sequence with the start character.
        target_seq[0, 0, target_token_index["<BOS>"]] = 1.0

        # Sampling loop for a batch of sequences
        # (to simplify, here we assume a batch of size 1).
        stop_condition = False
        decoded_sentence = []
        decoded_sentence_Bis = []
        while not stop_condition:
            output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        
            # Sample a token
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            sampled_char = reverse_target_char_index[sampled_token_index]

            decoded_sentence_Bis.append(sampled_char)
            #decoded_sentence.append(sampled_char)

            # Exit condition: either hit max length
            # or find stop character.
            if sampled_char == "<EOS>" or len(decoded_sentence) > max_decoder_seq_length:
                stop_condition = True

            # Update the target sequence (of length 1).
            target_seq = np.zeros((1, 1, num_decoder_tokens))
            target_seq[0, 0, sampled_token_index] = 1.0

            # Update states
            states_value = [h, c]

            if sampled_char != "<EOS>":
                l = list(np.argsort(output_tokens[0, -1, :]))
                l.reverse()
                l = l[:nMot]
                lis = []
                for i in l :
                    lis.append(reverse_target_char_index[i])
                decoded_sentence.append(lis)



        return decoded_sentence,decoded_sentence_Bis


    """
    You can now generate decoded sentences as such:
    """
    res = []

    for i in range(0,len(encoder_input_data)):
        # Take one sequence (part of the training set)
        # for trying out decoding.
        input_seq = encoder_input_data[i : i + 1]
        decoded_sentence,decoded_sentence_Bis = decode_sequence(input_seq)
        print(decoded_sentence_Bis)
        res.append(decoded_sentence)
    return res


res = returntab("testTrad (1).txt",10,num_encoder_tokens,input_token_index,target_token_index)
print(res)


ValueError: ignored

In [ ]:

from itertools import combinations, product

"""
Title: Character-level recurrent sequence-to-sequence model
Author: [fchollet](https://twitter.com/fchollet)
Date created: 2017/09/29
Last modified: 2020/04/26
Description: Character-level recurrent sequence-to-sequence model.
Accelerator: GPU
"""
"""
## Introduction

This example demonstrates how to implement a basic character-level
recurrent sequence-to-sequence model. We apply it to translating
short English sentences into short French sentences,
character-by-character. Note that it is fairly unusual to
do character-level machine translation, as word-level
models are more common in this domain.

**Summary of the algorithm**

- We start with input sequences from a domain (e.g. English sentences)
    and corresponding target sequences from another domain
    (e.g. French sentences).
- An encoder LSTM turns input sequences to 2 state vectors
    (we keep the last LSTM state and discard the outputs).
- A decoder LSTM is trained to turn the target sequences into
    the same sequence but offset by one timestep in the future,
    a training process called "teacher forcing" in this context.
    It uses as initial state the state vectors from the encoder.
    Effectively, the decoder learns to generate `targets[t+1...]`
    given `targets[...t]`, conditioned on the input sequence.
- In inference mode, when we want to decode unknown input sequences, we:
    - Encode the input sequence into state vectors
    - Start with a target sequence of size 1
        (just the start-of-sequence character)
    - Feed the state vectors and 1-char target sequence
        to the decoder to produce predictions for the next character
    - Sample the next character using these predictions
        (we simply use argmax).
    - Append the sampled character to the target sequence
    - Repeat until we generate the end-of-sequence character or we
        hit the character limit.
"""

"""
## Setup
"""


"""
## Download the data
"""

"""shell
!curl -O http://www.manythings.org/anki/fra-eng.zip
!unzip fra-eng.zip
"""

"""
## Configuration
"""




import re
import numpy as np
import tensorflow as tf
from tensorflow import keras
def returntab(test_path, nMot):

    batch_size = 64  # Batch size for training.
    epochs = 2  # Number of epochs to train for.
    latent_dim = 256  # Latent dimensionality of the encoding space.
    num_samples = 10000  # Number of samples to train on.
    # Path to the data txt file on disk.
    data_path = "fra.txt"

    """
    ## Prepare the data
    """

    # Vectorize the data.
    input_texts = []
    target_texts = []
    input_words = set()
    target_words = set()
    with open(data_path, "r", encoding="utf-8") as f:
        lines = f.read().split("\n")
    for line in lines[: min(num_samples, len(lines) - 1)]:
        input_text, target_text, _ = line.split("\t")
        input_text = [w for w in re.split(
            "[^a-z0-9áàÀâÂçÇéÉèêÊëîïôÔœùûü']", input_text.lower()) if w != '']
        target_text = ['<BOS>'] + [w for w in re.split(
            "[^a-z0-9áàÀâÂçÇéÉèêÊëîïôÔœùûü]", target_text.lower()) if w != ''] + ['<EOS>']
        input_texts.append(input_text)
        target_texts.append(target_text)
        for word in input_text:
            if word not in input_words:
                input_words.add(word)
        for word in target_text:
            if word not in target_words:
                target_words.add(word)

    input_wordsSorted = sorted(list(input_words))
    target_words = sorted(list(target_words))
    num_encoder_tokens = len(input_wordsSorted)
    num_decoder_tokens = len(target_words)
    max_encoder_seq_length = max([len(txt) for txt in input_texts])
    max_decoder_seq_length = max([len(txt) for txt in target_texts])

    print("Number of samples:", len(input_texts))
    print("Number of unique input tokens:", num_encoder_tokens)
    print("Number of unique output tokens:", num_decoder_tokens)
    print("Max sequence length for inputs:", max_encoder_seq_length)
    print("Max sequence length for outputs:", max_decoder_seq_length)

    input_token_index = dict([(char, i)
                             for i, char in enumerate(input_wordsSorted)])
    target_token_index = dict([(char, i)
                              for i, char in enumerate(target_words)])

    encoder_input_data = np.zeros(
        (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32"
    )
    decoder_input_data = np.zeros(
        (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
    )
    decoder_target_data = np.zeros(
        (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
    )

    for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.0
        # encoder_input_data[i, t + 1 :, input_token_index[" "]] = 1.0
        for t, char in enumerate(target_text):
            # decoder_target_data is ahead of decoder_input_data by one timestep
            decoder_input_data[i, t, target_token_index[char]] = 1.0
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.0

    """
    ## Build the model
    """

    # Define an input sequence and process it.
    encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
    encoder = keras.layers.LSTM(latent_dim, return_state=True)
    encoder_outputs, state_h, state_c = encoder(encoder_inputs)

    # We discard `encoder_outputs` and only keep the states.
    encoder_states = [state_h, state_c]

    # Set up the decoder, using `encoder_states` as initial state.
    decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

    # We set up our decoder to return full output sequences,
    # and to return internal states as well. We don't use the
    # return states in the training model, but we will use them in inference.
    decoder_lstm = keras.layers.LSTM(
        latent_dim, return_sequences=True, return_state=True)
    decoder_outputs, _, _ = decoder_lstm(
        decoder_inputs, initial_state=encoder_states)
    decoder_dense = keras.layers.Dense(
        num_decoder_tokens, activation="softmax")
    decoder_outputs = decoder_dense(decoder_outputs)

    # Define the model that will turn
    # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
    model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

    """
    ## Train the model
    """

    model.compile(
        optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
    )

    input_texts = []

    with open(test_path, "r", encoding="utf-8") as f:
        lines = f.read().split("\n")
    for line in lines:
        input_text = line
        input_text = [w for w in re.split(
            "[^a-z0-9áàÀâÂçÇéÉèêÊëîïôÔœùûü']", input_text.lower()) if w != '']
        input_texts.append(input_text)
        for word in input_text:
            if word not in input_words:
                # input_words.add(word)
                print("error :" + word)

    num_encoder_tokens = len(input_words)
    max_encoder_seq_length = max([len(txt) for txt in input_texts])

    input_token_index = dict([(char, i)
                             for i, char in enumerate(input_wordsSorted)])

    encoder_input_data = np.zeros(
        (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")

    for i, input_text in enumerate(input_texts):
        for t, char in enumerate(input_text):
            encoder_input_data[i, t, input_token_index[char]] = 1.0

    """
    ## Run inference (sampling)

    1. encode input and retrieve initial decoder state
    2. run one step of decoder with this initial state
    and a "start of sequence" token as target.
    Output will be the next target token.
    3. Repeat with the current target token and current states
    """

    # Define sampling models
    # Restore the model and construct the encoder and decoder.
    model = keras.models.load_model("s2s")

    encoder_inputs = model.input[0]  # input_1
    # lstm_1
    encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output
    encoder_states = [state_h_enc, state_c_enc]
    encoder_model = keras.Model(encoder_inputs, encoder_states)

    decoder_inputs = model.input[1]  # input_2
    decoder_state_input_h = keras.Input(shape=(latent_dim,))
    decoder_state_input_c = keras.Input(shape=(latent_dim,))
    decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
    decoder_lstm = model.layers[3]
    decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
        decoder_inputs, initial_state=decoder_states_inputs
    )
    decoder_states = [state_h_dec, state_c_dec]
    decoder_dense = model.layers[4]
    decoder_outputs = decoder_dense(decoder_outputs)
    decoder_model = keras.Model(
        [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
    )

    # Reverse-lookup token index to decode sequences back to
    # something readable.
    reverse_input_char_index = dict((i, char)
                                    for char, i in input_token_index.items())
    reverse_target_char_index = dict((i, char)
                                     for char, i in target_token_index.items())

    def decode_sequence(input_seq):
        # Encode the input as state vectors.
        states_value = encoder_model.predict(input_seq)

        # Generate empty target sequence of length 1.
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        # Populate the first character of target sequence with the start character.
        target_seq[0, 0, target_token_index["<BOS>"]] = 1.0

        # Sampling loop for a batch of sequences
        # (to simplify, here we assume a batch of size 1).
        stop_condition = False
        decoded_sentence = []
        while not stop_condition:
            output_tokens, h, c = decoder_model.predict(
                [target_seq] + states_value)

            # Sample a token
            sampled_token_index = np.argmax(output_tokens[0, -1, :])
            sampled_char = reverse_target_char_index[sampled_token_index]
            # decoded_sentence.append(sampled_char)

            # Exit condition: either hit max length
            # or find stop character.
            if sampled_char == "<EOS>" or len(decoded_sentence) > max_decoder_seq_length:
                stop_condition = True

            # Update the target sequence (of length 1).
            target_seq = np.zeros((1, 1, num_decoder_tokens))
            target_seq[0, 0, sampled_token_index] = 1.0

            # Update states
            states_value = [h, c]

            if sampled_char != "<EOS>":
                l = list(np.argsort(output_tokens[0, -1, :]))
                l.reverse()
                l = l[:nMot]
                lis = []
                for i in l:
                    lis.append(reverse_target_char_index[i])
                decoded_sentence.append(lis)

        return decoded_sentence

    """
    You can now generate decoded sentences as such:
    """
    res = []

    for i in range(0, len(encoder_input_data)):
        # Take one sequence (part of the training set)
        # for trying out decoding.
        input_seq = encoder_input_data[i: i + 1]
        decoded_sentence = decode_sequence(input_seq)
        res.append(decoded_sentence)
    return res


res = returntab("testTrad.txt", 5)

print(res)



def keep_first_strings(lists_of_strings):
    first_strings = []
    for sublist in lists_of_strings:
        first_strings.append(sublist[0])
    return first_strings

def replace_last_strings(lists_of_strings, replacement_strings):
    for i, sublist in enumerate(lists_of_strings):
        sublist[-1] = replacement_strings[i]
    return lists_of_strings



def parse_file_to_array(file_name):
    letters = []
    with open(file_name, 'r', encoding='utf-8') as f:
        for line in f:
            letters.extend(line.strip().split())
    return letters

letters = parse_file_to_array('./phonemes.txt')
print(letters)


def find_last_char_of_phonetic_of_line_ending_with_word(file_name, word, letters):
    with open(file_name, 'r', encoding='utf-8') as f:
        for line in f:
            parts = line.strip().split("/")
            if parts[0].strip().endswith(word):
              if parts[1][-2:] not in letters:
                return parts[1][-1]
              else:
                return parts[1][-2:] #return the last character of the phonetic
    return None

def tableau_phonetic(arr):
    i = 0
    resultat = []
    for sub_arr in arr:
        resultat = resultat + [(sub_arr[-1])]
    return resultat

def replace_phonetic(lists):
    return [[sublist,[find_last_char_of_phonetic_of_line_ending_with_word('./fr_FR.txt',x,letters) for x in sublist]] for sublist in lists]

def check_rime_structure(arrays, type_rime):
    array = arrays[1]
    if type_rime == 'AAAA':
        if array == [array[0]]*4:
            return arrays[0]
    elif type_rime == 'AABB':
        if array[:2] == [array[0]]*2 and array[2:] == [array[2]]*2:
            return arrays[0]
    elif type_rime == 'ABAB':
        if array[0] == array[2] and array[1] == array[3]:
            return arrays[0]
    else:
        return False


def find_phonetic_combinations(array, type_rime):
    int_lists = tableau_phonetic(array)
    combinations = list(product(*int_lists))
    res = replace_phonetic(combinations)
    resultat=[]
    for i in res:
      if check_rime_structure(i,type_rime):
        resultat = resultat + i
    temp = keep_first_strings(array)
    print(len(resultat))
    resultat = replace_last_strings(temp,resultat[0])
    return resultat
  

find_last_char_of_phonetic_of_line_ending_with_word('./fr_FR.txt', 'prix', letters)